<a href="https://colab.research.google.com/github/Dharani1999/Word-embedding-techniques/blob/master/Code_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora, models, similarities
from gensim.models import Word2Vec

In [2]:
max_seq_length = 100

In [3]:
ratings1 = pd.read_csv('/content/drive/My Drive/Movielensdata/ml25m/ratings.csv')
#print(len(ratings1['movieId'].unique()))
ratings = ratings1.iloc[0:1000,:]
#print(len(ratings['movieId'].unique()))
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


**Sorting the dataframe**

In [4]:
ratings.sort_values(by=['userId','timestamp'],inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
ratings.head(10)

,userId,movieId,rating,timestamp
36,1,5952,4.0,1147868053
13,1,2012,2.5,1147868068
12,1,2011,2.5,1147868079
11,1,1653,4.0,1147868097
9,1,1250,4.0,1147868414
40,1,6539,3.5,1147868461
39,1,6377,4.0,1147868469
21,1,3448,4.0,1147868480
5,1,1088,4.0,1147868495
4,1,899,3.5,1147868510


In [6]:
users_total = len(ratings['userId'].unique())
print(users_total)

4


**Selecting the most recent movies rated by each user and padding if necessary**

In [7]:
movie_list = []
for i in range(users_total):
  list1 = []
  list1 = ratings.loc[ratings['userId'] ==(i+1),['movieId']]['movieId'].tolist()
  if len(list1)>max_seq_length:
    list1 = list1[(len(list1)-max_seq_length):]
  elif len(list1)<max_seq_length:
    for j in range((max_seq_length-len(list1))):
      list1.append(0)
  movie_list.append(list1)
print(movie_list)

[[5952, 2012, 2011, 1653, 1250, 6539, 6377, 3448, 1088, 899, 4308, 2161, 6711, 3949, 8360, 5878, 306, 1175, 307, 1237, 7327, 8154, 7234, 2843, 4144, 7365, 2068, 4422, 4973, 6016, 8873, 2692, 27721, 7323, 6954, 8014, 7939, 6370, 8973, 4703, 31956, 5147, 8786, 1260, 2351, 7940, 7209, 8685, 7820, 7937, 7938, 8405, 4325, 2632, 1217, 8729, 5912, 5767, 665, 2573, 27266, 8327, 32591, 5269, 3569, 27193, 5684, 7318, 296, 7361, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4306, 8368, 8360, 3793, 4995, 4963, 8636, 2355, 8665, 2571, 589, 33493, 7153, 1873, 1246, 1584, 3994, 2139, 31923, 4720, 2294, 2745, 2138, 858, 33660, 2268, 2501, 6539, 1270, 6565, 4535, 1198, 1302, 349, 3098, 1907, 1580, 364, 480, 3360, 524, 5103, 62, 7075, 1287, 260, 318, 588, 1693, 1196, 4993, 2918, 3105, 5955, 1275, 1393, 6879, 5010, 6311, 8970, 1356, 3175, 4016, 1672, 1257, 2115, 36527, 1674, 1376, 7624, 2359, 8958, 35836, 2761, 953, 1923, 6947, 3148, 2083, 8010, 7162, 914, 40

**Selecting the most recent ratings of each user and padding if necessary**

In [8]:
rating_list =[]
for i in range(users_total):
  list2 = []
  list2 = ratings.loc[ratings['userId'] ==(i+1),['rating']]['rating'].tolist()
  if len(list2)>max_seq_length:
    list2 = list2[(len(list2)-max_seq_length):]
  elif len(list2)<max_seq_length:
    for j in range((max_seq_length-len(list2))):
      list2.append(0)
  rating_list.append(list2)
print(rating_list)

[[4.0, 2.5, 2.5, 4.0, 4.0, 3.5, 4.0, 4.0, 4.0, 3.5, 3.0, 3.5, 5.0, 5.0, 4.0, 4.0, 3.5, 3.5, 5.0, 5.0, 3.5, 5.0, 4.5, 4.5, 5.0, 4.0, 2.5, 3.0, 4.5, 5.0, 3.0, 5.0, 3.0, 3.5, 3.5, 3.5, 2.5, 4.5, 4.0, 4.0, 3.5, 4.0, 4.0, 3.5, 4.5, 4.5, 4.0, 1.0, 2.5, 3.0, 2.5, 3.5, 5.0, 5.0, 3.5, 3.5, 3.0, 5.0, 5.0, 4.0, 4.5, 5.0, 5.0, 0.5, 5.0, 3.0, 2.0, 2.0, 5.0, 5.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4.5, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 4.5, 4.0, 5.0, 4.0, 4.5, 5.0, 4.0, 4.0, 5.0, 3.0, 4.0, 3.5, 4.0, 4.5, 5.0, 4.5, 3.5, 5.0, 4.5, 4.5, 5.0, 4.0, 3.0, 3.0, 4.0, 3.0, 4.5, 3.0, 4.0, 4.5, 4.5, 2.0, 5.0, 3.5, 5.0, 0.5, 5.0, 5.0, 5.0, 5.0, 2.0, 4.0, 5.0, 5.0, 4.5, 4.0, 4.0, 4.5, 4.0, 5.0, 2.0, 5.0, 5.0, 5.0, 3.5, 5.0, 4.5, 5.0, 3.5, 3.5, 4.0, 5.0, 3.5, 5.0, 2.5, 0.5, 0.5, 4.5, 0.5, 4.0, 2.0, 4.5, 5.0, 0.5, 4.0, 3.0, 0.5, 4.5, 3.0, 1.5, 1.5, 0.5, 5.0, 4.5, 4.5, 4.5, 0.5, 3.0, 0.5, 5.0, 2.0, 5.0, 5.0], [4.5, 3.0, 4.0, 4.0, 3.0, 4.0, 3.5, 4.0, 4.5, 3.5, 3.5,

In [9]:
movies_transpose = pd.DataFrame(data=movie_list,index=[i+1 for i in range(users_total)])
movies_transpose.index.names = ['userId']
print(movies_transpose)

            0     1     2       3     4   ...     95      96     97    98    99
userId                                    ...                                  
1         5952  2012  2011    1653  1250  ...      0       0      0     0     0
2         4306  8368  8360    3793  4995  ...   2987    1293   7090  1136  2150
3       176371  3968  8983  102407  6548  ...  88125  106489  81834  4262  1732
4         8665  8961  4993    1580  5444  ...      0       0      0     0     0

[4 rows x 100 columns]


In [10]:
ratings_transpose = pd.DataFrame(data=rating_list,index=[i+1 for i in range(users_total)])
ratings_transpose.index.names = ['userId']
print(ratings_transpose)

         0    1    2    3    4    5    6   ...   93   94   95   96   97   98   99
userId                                     ...                                   
1       4.0  2.5  2.5  4.0  4.0  3.5  4.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2       4.5  5.0  4.0  5.0  5.0  4.0  5.0  ...  0.5  3.0  0.5  5.0  2.0  5.0  5.0
3       4.5  3.0  4.0  4.0  3.0  4.0  3.5  ...  2.5  4.0  3.5  3.5  3.5  4.5  4.5
4       3.5  5.0  4.5  4.5  4.0  4.0  4.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[4 rows x 100 columns]


In [11]:
# Select features from original dataset to form a new dataframe 
df1 = ratings_transpose.iloc[:]# For each row, combine all the columns into one column
df2 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)# Store them in a pandas dataframe
df_clean = pd.DataFrame({'clean': df2})# Create the list of list format of the custom corpus for gensim modeling 
sent = [row.split(',') for row in df_clean['clean']]

In [12]:
#list of lists format of the ratings transpose matrix
print(sent)

[['4.0', '2.5', '2.5', '4.0', '4.0', '3.5', '4.0', '4.0', '4.0', '3.5', '3.0', '3.5', '5.0', '5.0', '4.0', '4.0', '3.5', '3.5', '5.0', '5.0', '3.5', '5.0', '4.5', '4.5', '5.0', '4.0', '2.5', '3.0', '4.5', '5.0', '3.0', '5.0', '3.0', '3.5', '3.5', '3.5', '2.5', '4.5', '4.0', '4.0', '3.5', '4.0', '4.0', '3.5', '4.5', '4.5', '4.0', '1.0', '2.5', '3.0', '2.5', '3.5', '5.0', '5.0', '3.5', '3.5', '3.0', '5.0', '5.0', '4.0', '4.5', '5.0', '5.0', '0.5', '5.0', '3.0', '2.0', '2.0', '5.0', '5.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0'], ['4.5', '5.0', '4.0', '5.0', '5.0', '4.0', '5.0', '4.5', '4.0', '5.0', '4.0', '4.5', '5.0', '4.0', '4.0', '5.0', '3.0', '4.0', '3.5', '4.0', '4.5', '5.0', '4.5', '3.5', '5.0', '4.5', '4.5', '5.0', '4.0', '3.0', '3.0', '4.0', '3.0', '4.5', '3.0', '4.0', '4.5', '4.5', '2.0', '5.0', '3.5', '5.0', '0

In [13]:
model = Word2Vec(sent,min_count=1,size= 50,workers=3, window =3, sg = 1)

In [14]:
#user_vec = []
#for i in range(users_total):
#  a = np.zeros((50,))
#  for j in range(max_seq_length):
#    a = a+ model[sent[i][j]]
#  user_vec.append(a)

In [15]:
#user_vec

In [16]:
model.save('wordtovec')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [17]:
corpus = gensim.matutils.Dense2Corpus(np.array(sent))
print(list(corpus))
total_topics = 10

[[(0, 4.0), (1, 4.5), (2, 4.5), (3, 3.5)], [(0, 2.5), (1, 5.0), (2, 3.0), (3, 5.0)], [(0, 2.5), (1, 4.0), (2, 4.0), (3, 4.5)], [(0, 4.0), (1, 5.0), (2, 4.0), (3, 4.5)], [(0, 4.0), (1, 5.0), (2, 3.0), (3, 4.0)], [(0, 3.5), (1, 4.0), (2, 4.0), (3, 4.0)], [(0, 4.0), (1, 5.0), (2, 3.5), (3, 4.0)], [(0, 4.0), (1, 4.5), (2, 4.0), (3, 3.5)], [(0, 4.0), (1, 4.0), (2, 4.5), (3, 4.5)], [(0, 3.5), (1, 5.0), (2, 3.5), (3, 3.0)], [(0, 3.0), (1, 4.0), (2, 3.5), (3, 3.5)], [(0, 3.5), (1, 4.5), (2, 3.5), (3, 3.5)], [(0, 5.0), (1, 5.0), (2, 4.0), (3, 2.0)], [(0, 5.0), (1, 4.0), (2, 3.5), (3, 3.5)], [(0, 4.0), (1, 4.0), (2, 3.5), (3, 2.5)], [(0, 4.0), (1, 5.0), (2, 4.0), (3, 4.5)], [(0, 3.5), (1, 3.0), (2, 3.0), (3, 4.0)], [(0, 3.5), (1, 4.0), (2, 2.5), (3, 4.0)], [(0, 5.0), (1, 3.5), (2, 3.5), (3, 3.5)], [(0, 5.0), (1, 4.0), (2, 3.5), (3, 5.0)], [(0, 3.5), (1, 4.5), (2, 4.0), (3, 4.0)], [(0, 5.0), (1, 5.0), (2, 3.5), (3, 3.5)], [(0, 4.5), (1, 4.5), (2, 4.0), (3, 4.5)], [(0, 4.5), (1, 3.5), (2, 4.0), (3

In [18]:
lsi = models.LsiModel(corpus, num_topics=total_topics)
index = similarities.MatrixSimilarity(lsi[corpus])
lsi_data = lsi[corpus]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [19]:
#list(lsi_data)

In [20]:
lsi_topics = lsi.print_topics()
for topic in lsi_topics:
  print(topic)

(0, '0.574*"1" + 0.491*"3" + 0.491*"2" + 0.435*"0"')
(1, '0.876*"0" + -0.420*"2" + -0.213*"1" + -0.107*"3"')
(2, '-0.757*"3" + 0.651*"1" + 0.045*"0" + -0.043*"2"')
(3, '-0.762*"2" + 0.450*"1" + 0.418*"3" + -0.205*"0"')


In [21]:
lsi.save('lsimodel')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [22]:
tfidf = models.TfidfModel(corpus)
tfidf_data = tfidf[corpus]
print(list(tfidf_data))

[[(0, 0.9681818217953104), (3, 0.2502477970834385)], [(0, 0.860970331831548), (3, 0.508655175640506)], [(0, 0.8829460572852806), (3, 0.4694744507684928)], [(0, 0.9489711017505389), (3, 0.3153630416557533)], [(0, 0.9590328439756357), (3, 0.28329490672443064)], [(0, 0.9474647299913281), (3, 0.3198602592108932)], [(0, 0.9590328439756357), (3, 0.28329490672443064)], [(0, 0.9681818217953104), (3, 0.2502477970834385)], [(0, 0.9489711017505389), (3, 0.3153630416557533)], [(0, 0.9694088994824559), (3, 0.24545139153040807)], [(0, 0.9454307688329421), (3, 0.32582305219850843)], [(0, 0.9590328439756358), (3, 0.28329490672443064)], [(0, 0.9930915301499692), (3, 0.11734228881521185)], [(0, 0.9792835398786688), (3, 0.20249382341865033)], [(0, 0.9833808807323089), (3, 0.18155451911243758)], [(0, 0.9489711017505389), (3, 0.3153630416557533)], [(0, 0.9474647299913281), (3, 0.3198602592108932)], [(0, 0.9474647299913281), (3, 0.3198602592108932)], [(0, 0.9792835398786688), (3, 0.20249382341865033)], [(0,

In [23]:
#lsi_list = np.zeros((len(lsi_data), len(lsi_topics)), dtype=np.float64)
#for i in range(len(corpus)):
#    for j in range(len(lsi_topics)):
#        lsi_list[i][j]= list(lsi_data)[i][j][1]

tfidf_token= np.zeros((len(tfidf_data), 350), dtype=np.float64)
tfidf_vals= np.zeros((len(tfidf_data), 350), dtype=np.float64)

for i in range(len(corpus)):
    for k in range(len(list(tfidf_data)[i])):
        tfidf_token[i][k]=(list(tfidf_data))[i][k][0]
        tfidf_vals[i][k]=(list(tfidf_data))[i][k][1]

tfidf_list=list(tfidf_data)

In [24]:
tfidf_list[1]

[(0, 0.860970331831548), (3, 0.508655175640506)]